In [1]:
import sys
import os

# FIX: Resolver conflicto de OpenMP (Error #15) que causa crash del kernel
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import json
import shutil
import types
import importlib
from gymnasium import spaces

# Configuración de rutas locales
project_path = os.getcwd()
if project_path not in sys.path:
    sys.path.append(project_path)

baselines_path = os.path.join(project_path, 'baselines')
if baselines_path not in sys.path:
    sys.path.append(baselines_path)

print(f"Directorio de trabajo: {project_path}")

Directorio de trabajo: c:\Users\javi1\Documents\repos_git\TEL351-PokemonRed


In [2]:
# --- RELOAD MODULES ---
def reload_modules():
    modules_to_reload = [
        'v2.red_gym_env_v2',
        'advanced_agents.features',
        'advanced_agents.wrappers',
        'advanced_agents.base',
        'advanced_agents.train_agents',
        'advanced_agents.combat_apex_agent',
        'advanced_agents.puzzle_speed_agent',
        'advanced_agents.hybrid_sage_agent',
        'advanced_agents.transition_models'
    ]
    for mod_name in modules_to_reload:
        if mod_name in sys.modules:
            try:
                importlib.reload(sys.modules[mod_name])
                print(f"♻️ Recargado: {mod_name}")
            except Exception as e:
                print(f"⚠️ No se pudo recargar {mod_name}: {e}")

reload_modules()

In [3]:
# Copiar events.json si es necesario
events_source = os.path.join(project_path, 'baselines', 'events.json')
events_dest = os.path.join(project_path, 'events.json')
if os.path.exists(events_source) and not os.path.exists(events_dest):
    shutil.copy(events_source, events_dest)
    print(f"Copiado events.json a {events_dest}")

In [4]:
try:
    from advanced_agents.train_agents import _base_env_config
    from advanced_agents.combat_apex_agent import CombatApexAgent, CombatAgentConfig
    from advanced_agents.puzzle_speed_agent import PuzzleSpeedAgent, PuzzleAgentConfig
    from advanced_agents.hybrid_sage_agent import HybridSageAgent, HybridAgentConfig
except ImportError as e:
    print("ERROR CRÍTICO: Fallo en imports.")
    raise e

# --- CONFIGURACIÓN ---
AGENT_TYPE = "combat"
SCENARIO_ID = "pewter_brock"
PHASE_NAME = "battle"
TOTAL_TIMESTEPS = 100000
# ---------------------

# Cargar configuración del escenario
with open('gym_scenarios/scenarios.json', 'r') as f:
    scenarios_data = json.load(f)

scenario = next((s for s in scenarios_data['scenarios'] if s['id'] == SCENARIO_ID), None)
if not scenario:
    raise ValueError(f"Escenario {SCENARIO_ID} no encontrado")

phase = next((p for p in scenario['phases'] if p['name'] == PHASE_NAME), None)
if not phase:
    raise ValueError(f"Fase {PHASE_NAME} no encontrada en {SCENARIO_ID}")

state_file_path = phase['state_file']
print(f"Entrenando {AGENT_TYPE} en {SCENARIO_ID} - {PHASE_NAME}")

# Fallback de estado
if not os.path.exists(state_file_path):
    possible_fallbacks = ['init.state', 'has_pokedex.state', 'has_pokedex_nballs.state', 'fast_text_start.state']
    available_fallbacks = [p for p in possible_fallbacks if os.path.exists(p)]
    if available_fallbacks:
        state_file_path = available_fallbacks[0]
        print(f"Usando fallback: {state_file_path}")
    else:
        raise FileNotFoundError("No se encontró archivo de estado inicial.")

# Configuración del entorno LOCAL
env_overrides = {
    "init_state": state_file_path,
    "headless": False,  # <--- IMPORTANTE: False para ver la ventana del emulador
    "save_video": False,
    "gb_path": "PokemonRed.gb"
}

def train_local(agent_cls, config_cls):
    config = config_cls(
        env_config=_base_env_config(env_overrides), 
        total_timesteps=TOTAL_TIMESTEPS
    )
    agent = agent_cls(config)
    
    # Check Dict observation space
    env_for_check = agent.make_env()
    obs_space = getattr(env_for_check, 'observation_space', None)
    if isinstance(obs_space, spaces.Dict):
        print("Observación Dict detectada -> MultiInputPolicy")
        agent.policy_name = types.MethodType(lambda self: "MultiInputPolicy", agent)

    return agent.train()

if AGENT_TYPE == "combat":
    runtime = train_local(CombatApexAgent, CombatAgentConfig)
elif AGENT_TYPE == "puzzle":
    runtime = train_local(PuzzleSpeedAgent, PuzzleAgentConfig)
elif AGENT_TYPE == "hybrid":
    runtime = train_local(HybridSageAgent, HybridAgentConfig)

print("Entrenamiento finalizado.")
model = runtime.model

Entrenando combat en pewter_brock - battle
Observación Dict detectada -> MultiInputPolicy
Observación Dict detectada -> MultiInputPolicy


KeyboardInterrupt: 

In [ ]:
# Guardar modelo
save_dir = "models_local"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, f"{AGENT_TYPE}_{SCENARIO_ID}_{PHASE_NAME}")
model.save(save_path)
print(f"Modelo guardado en {save_path}")